In [190]:
# Import 
import pandas as pd
import numpy as np
from scipy import signal

In [191]:
# Input file
folder_path = r'D:\iriwa\デスクトップ\Code\Project_Python\Research'
file_name = r'\yoshiyama.csv'
output_filename = '吉山'+'(Eye)'

input_data = pd.read_csv(folder_path + file_name, header=None, skiprows=1,sep=';',names=('No.','timestamp','eyelids_gap','pupil_x','pupil_y','pupil_diameter'))

input_data = input_data.dropna()
print(input_data)

No.                timestamp  eyelids_gap  pupil_x  pupil_y  \
0            1  24-11-2020 14:51:09.687         85.0    224.0    116.0   
1            2  24-11-2020 14:51:09.696         82.0    221.0    118.0   
2            3  24-11-2020 14:51:09.704         83.0    226.0    114.0   
3            4  24-11-2020 14:51:09.712         84.0    219.0    117.0   
4            5  24-11-2020 14:51:09.721         85.0    220.0    118.0   
...        ...                      ...          ...      ...      ...   
221784  221785  24-11-2020 15:21:55.334         41.0    209.0    106.0   
221785  221786  24-11-2020 15:21:55.343         43.0    209.0    106.0   
221786  221787  24-11-2020 15:21:55.352         41.0    209.0    107.0   
221787  221788  24-11-2020 15:21:55.359         44.0    209.0    107.0   
221788  221789  24-11-2020 15:21:55.368         41.0    209.0    107.0   

        pupil_diameter  
0                 37.0  
1                 37.0  
2                 36.0  
3                 39.0

In [192]:
N=128
fs = 120
dt = 1/fs
n = int(len(input_data))
time1 = pd.DataFrame({'time': [i*dt for i in range(n)]})
TIME = int(n/fs)
time2 = pd.DataFrame({'time': [i for i in range(TIME)]})
print(TIME)

1848


In [193]:
df = pd.DataFrame({'pupil': [i if i>5 else np.nan for i in input_data['pupil_diameter']]})
df = df.interpolate()
print(df)
df_1s = pd.DataFrame({'pupil':[df.iloc[i*fs:i*fs+fs,0].mean() for i in range(TIME)]})
print(df_1s)
        


pupil
0        37.0
1        37.0
2        36.0
3        39.0
4        36.0
...       ...
221784   30.0
221785   30.0
221786   30.0
221787   30.0
221788   30.0

[221789 rows x 1 columns]
          pupil
0     37.191667
1     37.691667
2     34.891667
3     38.000000
4     38.316667
...         ...
1843  33.191667
1844  33.091667
1845  33.000000
1846  30.391667
1847  28.000000

[1848 rows x 1 columns]


In [194]:
wf = signal.hamming(N)
MPF   = [np.nan for i in range(TIME)]

df2 = [i for i in df['pupil']]
print(df2)

[37.0, 37.0, 36.0, 39.0, 36.0, 36.0, 35.0, 36.0, 36.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 36.0, 36.0, 36.0, 36.0, 36.0, 36.0, 36.0, 37.0, 36.0, 37.0, 38.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 38.0, 37.0, 37.0, 36.0, 38.0, 38.0, 37.0, 37.0, 37.0, 37.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 37.0, 37.0, 36.0, 37.0, 37.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 37.0, 37.0, 37.0, 38.0, 38.0, 38.0, 39.0, 39.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 39.0, 38.0, 38.0, 38.0, 39.0, 38.0, 39.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 38.0, 37.0, 37.0, 37.0, 37.0, 37.0, 38.0, 37.0, 37.0, 37.0, 38.0, 37.0, 37.0, 36.0, 36.0, 37.0, 37.0, 37.0, 36.0, 36.0, 36.0, 37.0, 37.0, 38.0, 37.0, 37.0, 36.0, 37.0, 37.0, 37.0, 36.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 36.0, 37.0, 36.0, 36.0, 36.0, 36.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 36.0, 37.0, 36.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 37.0, 36.0, 37.0, 37.0, 38.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.

In [195]:
for i in range(TIME):
    # Formatted to 1s(1000+24data) & Apply window function  # 1秒(1000+24個)を切り取り & 窓関数適用
    Format_data = [df2[j+i*int(fs)] * wf[j] for j in range(N)]  # Formatted to 1s(1000+24data)  # 1秒(1000+24個)を切り取り

    #Func_FFT() #FFT    #高速フーリエ変換
    F = np.fft.fft(Format_data)
    amplitude = np.abs(F)   # Amplitude     # 振幅スペクトル(F/N/2)
    power = amplitude ** 2        # Power         # パワースペクトル
    density = power/(fs/N)     # Power Density # パワースペクトル密度
    freq = np.fft.fftfreq(N, d=dt)
    f_l = int(len(freq)/2)
    
    #Func_cal
    total_fp = 0
    total_power = 0

    for l in range(f_l):
        if freq[l] >= 1.0 and freq[l] <= 30: 
            total_power += amplitude[l]
            total_fp = total_fp + (amplitude[l] * freq[l])

    MPF[i]   = total_fp  / total_power
df_MPF = pd.DataFrame({'MPF': [i for i in MPF]})
print(df_MPF)

MPF
0     13.077490
1      8.868545
2     13.396885
3     13.569543
4     12.826032
...         ...
1843   9.369022
1844   7.299394
1845   4.248403
1846  11.379801
1847   4.248403

[1848 rows x 1 columns]


In [196]:
output_data = pd.concat([time2, df_1s,df_MPF], axis=1)
print(output_data)


time      pupil        MPF
0        0  37.191667  13.077490
1        1  37.691667   8.868545
2        2  34.891667  13.396885
3        3  38.000000  13.569543
4        4  38.316667  12.826032
...    ...        ...        ...
1843  1843  33.191667   9.369022
1844  1844  33.091667   7.299394
1845  1845  33.000000   4.248403
1846  1846  30.391667  11.379801
1847  1847  28.000000   4.248403

[1848 rows x 3 columns]


In [197]:
# Output data
output_folder_path = 'D:\\iriwa\\デスクトップ\\Code\\Project_Python\\Research\\Eye'
output_data.to_excel(output_folder_path + '\\' + output_filename +'_pupil.xlsx', sheet_name=output_filename, index=False)
